In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Загрузка и предварительная обработка данных
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(data_url, names=column_names, na_values="?", comment="\t", sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()
dataset = dataset.dropna()

# Разделение данных на тренировочную и тестовую выборки
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Извлечение меток и нормализация данных
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

# Создание модели с использованием TensorFlow
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=[len(train_dataset[0])]),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    return model

model = build_model()

# Компиляция и обучение модели
optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

history = model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=0)

# Оценка модели с использованием метрики MAE
predictions = model.predict(test_dataset)
mae = mean_absolute_error(test_labels, predictions)
print(f"MAE: {mae}")

3/3 [==============================] - 0s 5ms/step
MAE: 1.8206842253479778


In [8]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 5.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras_tuner import RandomSearch

# Загрузка и предварительная обработка данных
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(data_url, names=column_names, na_values="?", comment="\t", sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()
dataset = dataset.dropna()

# Разделение данных на тренировочную и тестовую выборки
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Извлечение меток и нормализация данных
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

# Создание модели с использованием TensorFlow
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=[len(train_dataset[0])]))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(tf.keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu'))

    model.add(tf.keras.layers.Dense(1))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

    return model

# Подбор оптимальных гиперпараметров
tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=30,
    executions_per_trial=3,
    directory='auto_mpg_tuning',
    project_name='mpg')

tuner.search_space_summary()

tuner.search(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=0)

# Получение лучшей модели
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Оценка модели с использованием метрики MAE
predictions = best_model.predict(test_dataset)
mae = mean_absolute_error(test_labels, predictions)
print(f"MAE: {mae}")

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from kerastuner.tuners import RandomSearch

# Загрузка и предварительная обработка данных
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(data_url, names=column_names, na_values="?", comment="\t", sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()
dataset = dataset.dropna()

# Разделение данных на тренировочную и тестовую выборки
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Извлечение меток и нормализация данных
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

# Создание модели с использованием TensorFlow
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=[len(train_dataset[0])]))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(tf.keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu'))

    model.add(tf.keras.layers.Dense(1))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

    return model

# Подбор оптимальных гиперпараметров
tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=30,
    executions_per_trial=3,
    directory='auto_mpg_tuning',
    project_name='mpg')

tuner.search_space_summary()

tuner.search(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=0)

# Получение лучшей модели
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Оценка модели с использованием метрики MAE
predictions = best_model.predict(test_dataset)
mae = mean_absolute_error(test_labels, predictions)
print(f"MAE: {mae}")
